In [1]:
# Importing dependencies
import pandas as pd
from datetime import datetime

In [2]:
# Establishing path to our aqi data file
aqi_path = "Resources/ad_viz_plotval_data.csv"

# Establishing path for US cases
cases_csvpath = "Resources/covid-19-data-master/us-counties.csv"

In [3]:
# Reading CSV files
aqi_data = pd.read_csv(aqi_path)
cases_data = pd.read_csv(cases_csvpath)

In [4]:
#Removing first column (index)
aqi_data = aqi_data.drop(['Site ID', 'POC', 'SITE_LATITUDE', 'SITE_LONGITUDE', 'STATE_CODE', 
                          'CBSA_CODE', 'Source','DAILY_OBS_COUNT','PERCENT_COMPLETE','AQS_PARAMETER_CODE', 
                          'AQS_PARAMETER_DESC'], axis=1)

In [5]:
# Reformatting Date values to match other data sets
date = []
for i in range(len(aqi_data['Date'])):
    new_date = datetime.strptime(aqi_data['Date'][i], '%m/%d/%Y').strftime('%Y-%m-%d')
    date.append(new_date)
aqi_data['date'] = date

In [6]:
# Dropping the incorrectly formated Date column
aqi_data = aqi_data.drop('Date', axis=1)

In [7]:
# Renaming Date and COUNTY column to match other data sets
aqi_data = aqi_data.rename(columns={'COUNTY':'county', 'COUNTY_CODE': 'fips'})

# Displaying the first 5 values of aqi_data
aqi_data.head()

,Daily Mean PM2.5 Concentration,UNITS,DAILY_AQI_VALUE,Site Name,CBSA_NAME,STATE,fips,county,date
0,1.9,ug/m3 LC,8,ALBANY COUNTY HEALTH DEPT,"Albany-Schenectady-Troy, NY",New York,1,Albany,2020-01-01
1,4.1,ug/m3 LC,17,ALBANY COUNTY HEALTH DEPT,"Albany-Schenectady-Troy, NY",New York,1,Albany,2020-01-01
2,9.1,ug/m3 LC,38,ALBANY COUNTY HEALTH DEPT,"Albany-Schenectady-Troy, NY",New York,1,Albany,2020-01-02
3,13.4,ug/m3 LC,54,ALBANY COUNTY HEALTH DEPT,"Albany-Schenectady-Troy, NY",New York,1,Albany,2020-01-03
4,12.9,ug/m3 LC,53,ALBANY COUNTY HEALTH DEPT,"Albany-Schenectady-Troy, NY",New York,1,Albany,2020-01-04


In [8]:
# Grouping by date and county then averaging the daily AQI values
avg_aqi = pd.DataFrame(aqi_data.groupby(['date', 'county', 'fips'])['DAILY_AQI_VALUE'].mean())

# Renaming columns
avg_aqi = avg_aqi.rename(columns = {'DAILY_AQI_VALUE':'Average AQI Value'})

# Displaying the first 5 values
avg_aqi.head()

Average AQI Value
date       county fips                   
2020-01-01 Albany 1             11.000000
           Bronx  5             20.400000
           Erie   29            34.000000
           Essex  31             0.000000
           Kings  47            22.666667

In [9]:
# Displaying first 5 rows of cases_data before filter
cases_data.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [10]:
# Filtering dataset to only display the state of New York
cases_data = cases_data.loc[cases_data['state'] == 'New York'].reset_index(drop = True)
cases_data.head(10)

,date,county,state,fips,cases,deaths
0,2020-03-01,New York City,New York,NaN,1,0
1,2020-03-02,New York City,New York,NaN,1,0
2,2020-03-03,New York City,New York,NaN,2,0
3,2020-03-04,New York City,New York,NaN,2,0
4,2020-03-04,Westchester,New York,36119.0,9,0
5,2020-03-05,Nassau,New York,36059.0,1,0
6,2020-03-05,New York City,New York,NaN,4,0
7,2020-03-05,Westchester,New York,36119.0,17,0
8,2020-03-06,Nassau,New York,36059.0,4,0
9,2020-03-06,New York City,New York,NaN,5,0


In [11]:
# Transferring results onto a CSV file
aqi_data.to_csv('Data Source/ny_counties_avg_aqi')

cases_data.to_csv('Data Source/ny_counties_cases.csv', index= False)

In [12]:
# Scraping Wiki
url ='https://simple.wikipedia.org/wiki/List_of_counties_in_New_York'
tables = pd.read_html(url)

In [13]:
# Establising columns
Newyork_df = tables[0]
Newyork_df.columns = ['County', 'FIPS Code', 'County Seat', 'Created', 'Formed from', 'Named for','Density', '2010 Population', 'Area', 'Map']

In [14]:
# Dropping columns
ny_df = Newyork_df.drop(['Created', 'Formed from', 'County Seat', 'Named for', 'Area', 'Map', '2010 Population'], axis = 1)

# Renaming the FIPS Code column to align with the ny cases table
ny_df = ny_df.rename(columns = {'FIPS Code': 'fips'})

# Displaying first 5 values
ny_df.head()

,County,fips,Density
0,Albany County,1,570.74
1,Allegany County,3,47.34
2,Bronx County,5,24118.20
3,Broome County,7,280.56
4,Cattaraugus County,9,61.31


In [15]:
# Transferring results onto a CSV file
ny_df.to_csv('Data Source/ny_density.csv', index= False)

In [16]:
final_df = pd.merge(avg_aqi, ny_df, on='fips', how='inner')
final_df.head()

,fips,Average AQI Value,County,Density
0,1,11.0,Albany County,570.74
1,1,38.0,Albany County,570.74
2,1,54.0,Albany County,570.74
3,1,53.0,Albany County,570.74
4,1,12.0,Albany County,570.74
